In [9]:
import os.path

import matplotlib
import matplotlib.pyplot as plt
from minineedle import needle
import pandas as pd
from tqdm import tqdm

from analysis.orthogroups import (
    HaematobiumClade,
    init_SEQUENCE_ID_map,
)

matplotlib.use('TkAgg')

WD_PATH = "data/from_MARS/OrthoFinder/Results_May10/WorkingDirectory/"


def do_needle(seqs):
    alg = needle.NeedlemanWunsch(*seqs)
    alg.align()
    return alg.get_identity()


seq_id_map = init_SEQUENCE_ID_map(WD_PATH)
inv_seq_id_map = {v: k for k, v in seq_id_map.items()}

sp0 = HaematobiumClade("bovis", "TD2_PRJEB44434")
sp1 = HaematobiumClade("curassoni", "PRJEB44434")

BLASTOUT = pd.read_csv(
    os.path.join(WD_PATH, "Blast0_1.txt"),
    names=("qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend",
                "sstart", "send", "evalue", "bitscore"),
    delimiter="\t"
)

"Infered" global alignment identity formula

In [27]:
def infer_global_pident(row, *lens):
    # return min(lens) / max(lens) * 100
    return row["length"] / max(lens) * 100

#### Criteria


In [4]:
MIN_BLAST_PIDENT = 100
MAX_PROT_LENGTH = 100
MAX_TEST_PIDENT = 90

#### Calculate residuals

In [28]:
df = BLASTOUT[BLASTOUT["pident"] >= MIN_BLAST_PIDENT]
residuals = []
observed = []
for idx, row in tqdm(df.iterrows(), total=len(df)):
    tid0 = inv_seq_id_map[row["qseqid"]]
    tid1 = inv_seq_id_map[row["sseqid"]]
    seqs = [sp0.get_protein_sequence(tid0), sp1.get_protein_sequence(tid1)]
    len0, len1 = map(len, seqs)
    est_glob_pid = infer_global_pident(row, len0, len1)
    if est_glob_pid < MAX_TEST_PIDENT and max(len0, len1) < MAX_PROT_LENGTH:
        glob_pid = do_needle(seqs)
        res = glob_pid - est_glob_pid
        residuals.append(res)
        observed.append(glob_pid)

100%|██████████| 4184/4184 [01:32<00:00, 45.14it/s]


#### Plot residuals

In [19]:
plot_path = f"plots/residual_la{MIN_BLAST_PIDENT}_ga{MAX_TEST_PIDENT}_pl{MAX_PROT_LENGTH}.png"
plt.scatter(observed, residuals)
plt.yticks(range(-50, 50, 10))
plt.ylim(-50, 50)
plt.grid()
plt.xlabel("Predicted global pident")
plt.ylabel("Residuals")
plt.axhline(y=0, color='r', linestyle='--')
plt.suptitle("Residual plot", fontsize=18)
plt.title(f'min BLAST pident: {MIN_BLAST_PIDENT}; max "infered" pident: {MAX_TEST_PIDENT}; max protein length: {MAX_PROT_LENGTH}',
             fontsize=11)
plt.savefig(plot_path)
plt.show()